In [184]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.patches import Rectangle #changed
import cv2  
import imageio
import pandas as pd
import seaborn as sns
import glob
import scipy.io
import os
from collections import Counter
from scipy.io.arff import loadarff 
from skimage.segmentation import mark_boundaries, slic

In [185]:
import ntpath
def path_leaf(path): # Get file name from path
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

In [205]:
GAZE_FOLDER_DIR = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_data/LUEBECK_GAZE/GazeCom_ground_truth/'
gaze_folder_list = [video for video in glob.glob(GAZE_DIR + '/*')]
gaze_folder_list.sort()


VIDEO_DIR = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_data/LUEBECK/'
video_path_list = [video for video in glob.glob(VIDEO_DIR + '/*')]
video_path_list.sort()

DETECT_VID_DIR = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results/LUEBECK/detect_video/'
detect_path_list = [video for video in glob.glob(DETECT_VID_DIR + '/*_0.AVI')]
detect_path_list.sort()

## Load Data

In [210]:
def load_df(gaze_path):
    
    
    from scipy.io.arff import loadarff
    arff = loadarff(gaze_path)
    df = pd.DataFrame.from_dict(arff[0])
    video_res_x = 1280
    video_res_y = 720
    df['time']-=df['time'][0]
    df['x'] = np.round(df['x'])
    df['y'] = np.round(df['y'])
    df = df[df['y'] >= 0]
    df = df[df['y'] < video_res_y]
    df = df[df['x'] >= 0]
    df = df[df['x'] < video_res_x]
    return df





In [212]:
df

,time,x,y,confidence,handlabeller1,handlabeller2,handlabeller_final
0,0.0,501.0,46.0,1.0,2.0,2.0,2.0
1,4000.0,501.0,46.0,1.0,2.0,2.0,2.0
2,8000.0,502.0,47.0,1.0,2.0,2.0,2.0
3,12000.0,510.0,53.0,1.0,2.0,2.0,2.0
4,16000.0,521.0,60.0,1.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...
5027,20228000.0,1065.0,380.0,1.0,1.0,1.0,1.0
5028,20232000.0,1065.0,381.0,1.0,1.0,1.0,1.0
5029,20236000.0,1065.0,380.0,1.0,1.0,1.0,1.0
5030,20240000.0,1065.0,380.0,1.0,1.0,1.0,1.0


In [221]:
#EVENTPATH = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_data/eye_tracking_data_tim/events_vid1-50_600-650/'
#translate_dict = {'NOISE':0, 'FIX':1, 'SP':2, 'SACCADE':3}
GAZEVID_DIR = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results/LUEBECK/gaze_video/'
translate_dict = {'UNKNOWN':0, 'FIX':1, 'SACCADE':2, 'SP':3, 'NOISE':4}

fps=30
fourcc = cv2.VideoWriter_fourcc(*'DIVX')

videos_run = [1]
subjects_run = [1] 

markersize = 50
maxframes = 100

for v in videos_run:#len(video_path_list):
    videoloc = detect_path_list[v] #video_path_list[v]
    vid = imageio.get_reader(videoloc,  'ffmpeg')
    vidlist = []
    
    gaze_dir = gaze_folder_list[v]
    gaze_path_list = [video for video in glob.glob(gaze_dir + '/*')]
    gaze_path_list.sort()
    
    for image in vid.iter_data():
        vidlist.append(np.array(image))
        
    for s in subjects_run:
        nframes = len(vidlist)
        gaze_path = gaze_path_list[s]
        video_name = os.path.splitext(path_leaf(gaze_path_list[s]))[0]
        df = load_df(gaze_path)
        
        vid_path = (GAZEVID_DIR + video_name + '_' + str(maxframes) + '.AVI')
        print(vid_path)
        # 10 fps, output format due to additional plot, usually (640,360)
        video = cv2.VideoWriter(vid_path, fourcc, 10, (1200,800))
       
        # go to either the full video or only through a certain number of frames
        for f in range(min(maxframes,len(vidlist))):
            dtemp = df[(df['time'] > 1e6*f/fps) & (df['time'] < 1e6*(f+1)/fps)] # assign data to frames
            fig, axs = plt.subplots(2, gridspec_kw={'height_ratios': [5, 1], 'wspace':0, 'hspace':0.05}, dpi=200)
            fig.suptitle(f'Video {v:03}, Subject {s:02}, Frame {f+1:04}')
            image = vidlist[f]
            axs[0].imshow(image)
            # filter the df for different events to plot them in different colors
            dfix = dtemp[dtemp['handlabeller_final']==1]
            dfix.plot.scatter(x='x', y='y', c='red', ax=axs[0], s=markersize, alpha=.3, marker='o')
            dsp = dtemp[dtemp['handlabeller_final']==3]
            dsp.plot.scatter(x='x', y='y', c='DarkBlue', ax=axs[0], s=markersize, alpha=.3, marker='o')
            dsac = dtemp[dtemp['handlabeller_final']==2]
            dsac.plot.scatter(x='x', y='y', c='DarkGreen', ax=axs[0], s=markersize, alpha=.8, marker='x')
            dnoise = dtemp[dtemp['handlabeller_final']==4]
            dnoise.plot.scatter(x='x', y='y', c='white', ax=axs[0], s=markersize, alpha=.8, marker='x')
            axs[0].axis('off')
            # plot where in the event-timeline we currently are
            #axs[1].plot(event_num)
            axs[1].plot(df['handlabeller_final'])
            axs[1].set_yticks([0,1,2,3,4]); axs[1].set_yticklabels(translate_dict.keys())
            axs[1].axvline(250/fps*f, color='red')
            axs[1].set_xticks([], [])
            
            canvas = FigureCanvas(fig)
            canvas.draw()
            width, height = fig.get_size_inches() * fig.get_dpi()
            temp = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3)  
            temp = cv2.cvtColor(temp,cv2.COLOR_RGB2BGR)

            video.write(temp)
            plt.close('all'); del temp
        video.release(); del video
        print('done!')

/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results/LUEBECK/gaze_video/AAW_breite_strasse_100.AVI


/Users/timschroder/env/lib/python3.7/site-packages/ipykernel_launcher.py:60: MatplotlibDeprecationWarning: Passing the minor parameter of set_xticks() positionally is deprecated since Matplotlib 3.2; the parameter will become keyword-only two minor releases later.


done!


In [156]:
os.path.splitext(path_leaf(gaze_path_list[v]))[0]

'AAF_breite_strasse'

In [158]:
GAZEVID_DIR

'/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results/LUEBECK/gaze_video/'